In [25]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()

roi = config.roi
data_folder = config.data_folder

age, _, _ = desired_lulc()

lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select([f"classification_{year}" for year in config.range_1985_2020])
        .byte()
        .rename([str(year) for year in config.range_1985_2020]))

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()

method = 'eu'
panama = True
if panama:
    roi = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_na", "equals", "Panama").geometry()
    folder = "panama"
else:
    folder = method

## Surrounding mature forests

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Mean fragmentation of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1)

In [18]:
def get_mature_mask():
    if method == 'eu':
        annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(roi)
        return annual_changes.select('Dec2020').eq(1).selfMask()
    elif method == 'mapbiomas':
        return lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()

def get_mature_biomass(biomass_raw, mature_mask):
    if method == 'eu':
        return biomass_raw.updateMask(mature_mask).rename("mature_biomass")
    elif method == 'mapbiomas':
        # keep only forests that are 1000m away from any non-forest area
        edge_detec = mature_mask.unmask(-1).zeroCrossing()
        distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
            .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
        distance_to_edge = distance_to_edge.gt(1000).selfMask().updateMask(mature_mask)
        return biomass_raw.updateMask(distance_to_edge).rename("mature_biomass")

def export_mature_biomass():
    age = ee.Image(f"{data_folder}/{folder}/secondary").select("age")
    biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
    proj = biomass_raw.projection().getInfo()

    mature_mask = get_mature_mask()
    sur_cover = mature_mask.unmask(0).focalMean(radius = 1000, units = "meters").float().updateMask(age).rename("sur_cover")
    mature_biomass = get_mature_biomass(biomass_raw, mature_mask)

    export_image(mature_biomass, "mature_biomass", region = roi, folder = folder, crsTransform = proj['transform'])
    export_image(mature_biomass, "mature_biomass_500", region = roi, folder = folder, scale = 500)
    export_image(sur_cover, "sur_cover", region = roi, folder = folder, scale = 30)

    return mature_biomass

export_mature_biomass()

In [27]:
def get_mature_biomass_vector(mature_biomass):
    # there are time out issues with larger mature_biomass images.
    # to get around it, we reduce to vectors over the geometries of the biomes we are considering
    # and we exclude the pixels that are within the given search radius of the biome edges
    if not panama:
        biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
        amaz_atla_pant = biome.filter(ee.Filter.inList('CD_Bioma', [1, 4, 6]))
        mature_biomass = mature_biomass.clip(amaz_atla_pant)

    mature_biomass_vector = mature_biomass.sample(region = mature_biomass.geometry(), scale = 1000, geometries = True)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = mature_biomass_vector,
        assetId = f"{data_folder}/{folder}/mature_biomass_vector",
        description = "mature_biomass_vector"
    )
        
    task.start()

mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass")
get_mature_biomass_vector(mature_biomass)


In [29]:
def get_distance_to_nearest_mature(method):

    secondary = ee.Image(f"{data_folder}/{folder}/secondary").select("age")

    mature_biomass_vector = ee.FeatureCollection(f"{data_folder}/{folder}/mature_biomass_vector")
    # change radius depending on data resolution and patterns
    distance_to_nearest_mature = mature_biomass_vector.distance(searchRadius = 10000)\
        .toInt().updateMask(secondary)

    export_image(distance_to_nearest_mature, "distance_to_nearest_mature", \
                 region = roi, folder = method, \
                    crsTransform = proj['transform'], scale = 1000)

    return distance_to_nearest_mature

distance_to_nearest_mature = get_distance_to_nearest_mature(method)


In [2]:
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000)))
    return buffered

def get_distances_vector_buffer(distance_to_nearest_mature, method):
    
    if panama:
        distance_to_nearest_mature = distance_to_nearest_mature \
        .addBands(distance_to_nearest_mature.gt(0).selfMask().rename("mask"))
        distances_vector = distance_to_nearest_mature.stratifiedSample(
            numPoints = 50000, classBand = "mask", region = roi, geometries = True
        )
    else:
        categorical = ee.Image(f"{data_folder}/categorical")
        biome_mask = categorical.select('biome').eq(1) \
        .Or(categorical.select('biome').eq(4)) \
        .Or(categorical.select('biome').eq(6))

        distance_to_nearest_mature = distance_to_nearest_mature \
            .addBands(biome_mask)

        distances_vector = distance_to_nearest_mature.stratifiedSample(
            numPoints = 50000, classBand = "biome", region = roi, geometries = True
        )

    distances_vector_buffer = distances_vector.map(buffer_distance)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = distances_vector_buffer,
        assetId = f"{data_folder}/{folder}/distances_vector_buffer",
        description = "distances_vector_buffer"
    )

    task.start()

distance_to_nearest_mature = ee.Image(f"{data_folder}/{folder}/distance_to_nearest_mature")
get_distances_vector_buffer(distance_to_nearest_mature, method)

In [4]:
def export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method):

    nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                            crs = proj['crs'], crsTransform = proj['transform'])

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = nearest_mature_biomass,
        assetId = f"{data_folder}/{folder}/nearest_mature_biomass1",
        description = "nearest_mature_biomass"
    )

    task.start()

mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass_500")
distances_vector_buffer = ee.FeatureCollection(f"{data_folder}/{folder}/distances_vector_buffer")
export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method)

In [5]:
def export_nearest_mature_biomass_image(nearest_mature_biomass, method):

    # change geometries from polygons to points
    nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

    # Convert the FeatureCollection to an image
    nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
        properties = ['first'],
        reducer = ee.Reducer.first()
    ).toInt16().rename('nearest_mature_biomass')

    export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image", \
                 folder = method, crsTransform = proj['transform'])

nearest_mature_biomass = ee.FeatureCollection(f"{data_folder}/{folder}/nearest_mature_biomass")
export_nearest_mature_biomass_image(nearest_mature_biomass, method)


In [28]:
mature_biomass = ee.Image(f"{data_folder}/{folder}/mature_biomass")

gaus_kernel = ee.Kernel.gaussian(radius = 1000, sigma = 200, units = "meters")
mat_gaus_ker = (
    mature_biomass.focalMean(
        radius = 1000, units = "meters", kernel = gaus_kernel, iterations = 1
    )
    .rename("mat_biomass")
    .float()
)

export_image(mat_gaus_ker, "mat_gaus_ker", region = roi, folder = method, crsTransform = proj['transform'])
